In [2]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:default:

In [3]:
%%sql
select 'hola mundo' 

,'hola mundo'
0,hola mundo


In [4]:
%%sql
create or replace table competencia_01 as 
select 
    *
from read_csv_auto('../datasets/competencia_02_crudo.csv')

,Success


In [24]:
%%sql 
SELECT column_name, data_type
FROM information_schema.columns
WHERE table_name = 'competencia_01';


,column_name,data_type
0,numero_de_cliente,BIGINT
1,foto_mes,BIGINT
2,active_quarter,BIGINT
3,cliente_vip,BIGINT
4,internet,BIGINT
...,...,...
149,Visa_fechaalta,BIGINT
150,Visa_mconsumototal,DOUBLE
151,Visa_cconsumos,BIGINT
152,Visa_cadelantosefectivo,BIGINT


In [30]:
%%sql
create or replace table competencia_02 as 

with max_db as 
    (
        select max(foto_mes) as max_foto_mes_db
        from competencia_01
    )
select c.*, max_db.max_foto_mes_db, max_numero_de_cliente.max_foto_mes, c.foto_mes,
    case
        when max_db.max_foto_mes_db = max_numero_de_cliente.max_foto_mes then 'continua'
        when CAST(SUBSTRING(max_numero_de_cliente.max_foto_mes, 1, 4) AS INTEGER) * 12 + CAST(SUBSTRING(max_numero_de_cliente.max_foto_mes, 5, 2) AS INTEGER) -
           (CAST(SUBSTRING(c.foto_mes, 1, 4) AS INTEGER) * 12 + CAST(SUBSTRING(c.foto_mes, 5, 2) AS INTEGER)) =  0 then 'baja+1'
        when CAST(SUBSTRING(max_numero_de_cliente.max_foto_mes, 1, 4) AS INTEGER) * 12 + CAST(SUBSTRING(max_numero_de_cliente.max_foto_mes, 5, 2) AS INTEGER) -
           (CAST(SUBSTRING(c.foto_mes, 1, 4) AS INTEGER) * 12 + CAST(SUBSTRING(c.foto_mes, 5, 2) AS INTEGER)) == 1 then 'baja+2'
         when CAST(SUBSTRING(max_numero_de_cliente.max_foto_mes, 1, 4) AS INTEGER) * 12 + CAST(SUBSTRING(max_numero_de_cliente.max_foto_mes, 5, 2) AS INTEGER) -
           (CAST(SUBSTRING(c.foto_mes, 1, 4) AS INTEGER) * 12 + CAST(SUBSTRING(c.foto_mes, 5, 2) AS INTEGER)) > 1 then 
            concat('baja+', CAST(SUBSTRING(max_numero_de_cliente.max_foto_mes, 1, 4) AS INTEGER) * 12 + CAST(SUBSTRING(max_numero_de_cliente.max_foto_mes, 5, 2) AS INTEGER) -
                            (CAST(SUBSTRING(c.foto_mes, 1, 4) AS INTEGER) * 12 + CAST(SUBSTRING(c.foto_mes, 5, 2) AS INTEGER)))
    end as clase_ternaria, 
    CAST(SUBSTRING(max_numero_de_cliente.max_foto_mes, 1, 4) AS INTEGER) * 12 + CAST(SUBSTRING(max_numero_de_cliente.max_foto_mes, 5, 2) AS INTEGER) -
        (CAST(SUBSTRING(c.foto_mes, 1, 4) AS INTEGER) * 12 + CAST(SUBSTRING(c.foto_mes, 5, 2) AS INTEGER)) as valor
from competencia_01 c
    cross join max_db
    left join 
        (
            select numero_de_cliente, max(foto_mes) as max_foto_mes
            from competencia_01
            group by numero_de_cliente
        ) max_numero_de_cliente on max_numero_de_cliente.numero_de_cliente = c.numero_de_cliente 

,Success


In [35]:
%%sql
select * from competencia_02
where numero_de_cliente = 181161368
order by foto_mes

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,max_foto_mes_db,max_foto_mes,foto_mes:1,clase_ternaria,valor
0,181161368,202011,1,0,0,29,3,-330.91,-346.94,1136.98,...,63,NaN,NaN,NaN,0.00,202107,202107,202011,continua,8
1,181161368,202012,1,0,0,29,4,-3304.76,-3651.71,1326.48,...,94,NaN,NaN,NaN,0.00,202107,202107,202012,continua,7
2,181161368,202101,1,0,0,29,5,-5199.17,-8850.88,22.67,...,125,0.0,0.0,0.0,785.91,202107,202107,202101,continua,6
3,181161368,202102,1,0,0,29,6,-4001.11,-12851.99,18.63,...,153,0.0,0.0,0.0,1700.85,202107,202107,202102,continua,5
4,181161368,202103,1,0,0,29,7,-3408.08,-16260.07,23.49,...,184,0.0,0.0,0.0,1700.85,202107,202107,202103,continua,4
5,181161368,202104,1,0,0,29,8,-2880.89,-19140.97,0.00,...,214,0.0,0.0,0.0,1700.85,202107,202107,202104,continua,3
6,181161368,202105,1,0,0,29,9,-2190.27,-21331.25,224.82,...,245,0.0,0.0,0.0,1700.85,202107,202107,202105,continua,2
7,181161368,202106,1,0,0,30,10,-1733.65,-23064.91,48.19,...,275,0.0,0.0,0.0,1700.85,202107,202107,202106,continua,1
8,181161368,202107,1,0,0,30,11,-560.62,-23625.53,113.68,...,306,0.0,0.0,0.0,1700.85,202107,202107,202107,continua,0
